# This is a notebook

We will show you how to call geoh5 projects from Analyst


In [13]:
import h5py
import numpy as np
import ipywidgets as widgets
from SimPEG import Utils
from library import Mag
#!/usr/bin/env python
# -*- coding: utf-8 -*-

In [14]:
# Load in GA h5
# work_dir ="C:\\Users\\DominiqueFournier\\Google Drive\\Research\\Tli_Kwi_Cho\\Modelling\\"
# fileName ="Test.geoh5"
work_dir = r'C:\Users\DominiqueFournier\Google Drive\Research\Tli_Kwi_Cho\Modelling'
fileName = r'\Consortium_Oct2017.geoh5'

In [15]:
# project = h5py.File(work_dir+fileName, 'r')

# keys = list(project['GEOSCIENCE']['Objects'].keys())

In [16]:
# list(project['GEOSCIENCE']['Data'][dataKeys[0]]['Data'])

In [17]:
# dataKeys = list(project['GEOSCIENCE']['Objects'][keys[0]]['Data'].keys())

# for dkey in dataKeys:
#     print(list(project['GEOSCIENCE']['Objects'][keys[0]]['Data'][dkey]['Data']))
    


In [18]:
# project.close()

In [19]:
def geoh5Reader(h5File):

    def listObjects(name):
        project = h5py.File(work_dir+fileName, 'r+')
        
        dictOut = {}
        for key in keys:
            if name == project['GEOSCIENCE']['Objects'][key].attrs['Name']:
                
#                 print(project['GEOSCIENCE']['Objects'][key])
                dictOut['x'] = project['GEOSCIENCE']['Objects'][key]['Vertices']['x']
                dictOut['y'] = project['GEOSCIENCE']['Objects'][key]['Vertices']['y']
                dictOut['z'] = project['GEOSCIENCE']['Objects'][key]['Vertices']['z']
                dkey = list(project['GEOSCIENCE']['Objects'][key]['Data'].keys())
                
                dictOut['data'] = np.r_[list(project['GEOSCIENCE']['Data'][dkey[5]]['Data'])]
                
                # Add random vector data
#                 project['GEOSCIENCE']['Objects'][key]['RandDataColmn'] = np.random.randn(loc_x.shape[0])
                
        project.close()
        return dictOut

    project = h5py.File(work_dir+fileName, 'r')

    keys = list(project['GEOSCIENCE']['Objects'].keys())

    names = []
    for key in keys:
        names += [project['GEOSCIENCE']['Objects'][key].attrs['Name']]

    project.close()
    out = widgets.interactive(listObjects,
                              name=widgets.Dropdown(
                                  options=names,
                                  value=names[0],
                                  description='Objects',
                                  disabled=False,
                                )
                             )
                              
    return out



In [20]:
out = geoh5Reader(work_dir+fileName)
display(out)


interactive(children=(Dropdown(description='Objects', options=('DIGHEM_Mag_floor10nt_25m', 'PK3', 'CDED_076c05…

In [21]:
survey = Mag.Mag.createMagSurvey(
    np.c_[out.result['x'],
          out.result['y'],
          out.result['z']], 
    EarthField=np.r_[50000, 90, 0],
    data=out.result['data']
) 

In [22]:
Mag.Simulator.dataHillsideWidget(survey)

interactive(children=(FloatSlider(value=0.0, continuous_update=False, description='SunAzimuth', max=360.0, ste…

In [23]:
sim = Mag.Simulator.ViewPrism(survey)
display(sim)

interactive(children=(ToggleButton(value=False, description='Refresh'), FloatSlider(value=351.25, continuous_u…

In [24]:
fwrSim = Mag.Simulator.PFSimulator(sim.result, survey)
display(fwrSim)

interactive(children=(ToggleButton(value=False, description='Refresh'), FloatSlider(value=0.1, continuous_upda…